In [ ]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
from helperToolz.evapo import *
from datetime import datetime
workhorse = True

if workhorse:
    origin = 'Aldhani/eoagritwin/'
else:
    origin = ''

def get_folders_in_dir(dir):
    return [f for f in os.listdir(dir) if os.path.isdir(os.path.join(dir, f))]

trash = '/data/Aldhani/eoagritwin/et/Auxiliary/trash/band_intermediate/'
temp = '/data/Aldhani/eoagritwin/et/Auxiliary/trash/vrt/'
os.makedirs(trash, exist_ok=True)
os.makedirs(temp, exist_ok=True)

In [ ]:
# read csv for valid row_cols for samples to draw. They are based on the share of agriculture (HR Landcover maps) within a S3 pixel
thresh_csv = pd.read_csv(f'/data/{origin}et/Auxiliary/landcover/csv/row_cols.csv')

for col in thresh_csv.columns:
    #print(f'finding indices for {col}')
    nested = [entry.split('_') for entry in thresh_csv[col] if type(entry) == str]
    rows, cols = zip(*nested)
    #print(len(rows))
# only use Thresh50, as all valid row_col will be here. For other THresh sets, a subset from this can be taken
# this are the last instances of rows and cols
rows = [int(row) for row in rows]
cols = [int(col) for col in cols]


In [ ]:
# check if all data is there and get the the different stacks (indices)
year = 2018
force_path = f'/data/Aldhani/eoagritwin/force/output/S3/{year}/'
folders = get_folders_in_dir(force_path)
folder_to_clean = [os.path.join(force_path, folder) for folder in folders if len(getFilelist(os.path.join(force_path, folder), '.tif', deep=True)) != 6]
if len(folder_to_clean) > 0:
    raise ValueError('the folders do not contain the same number of images...')
# for folder in folder_to_clean:
#     [os.remove(file) for file in getFilelist(folder, '.tif', deep=True) if '20181001' in file]
indices = list(set([file_annual.split('_')[-2] for file_annual in getFilelist(os.path.join(force_path, folders[0]), '.tif', deep=True)]))
indices.sort()


In [ ]:
# find the bands in raster stacks that belong to the year of interest
ds = gdal.Open(getFilelist(os.path.join(force_path, folders[0]), '.tif', deep=True)[0], 0)
if ds.RasterCount > 366:
    bandname = str(ds.GetRasterBand(93).GetDescription())
    if bandname == f'{year}0101':
        start_index = 93
        end_index = 457
        if year in [2016,2020,2024]:
            end_index +=1
    else:
        raise ValueError('band for Jan 01 not found!')

files = [getFilelist(os.path.join(force_path, folder), '.tif', deep=True) for folder in folders]
files = [file for list in files for file in list]

In [ ]:
#for index in indices:
index = indices[1]
index_files = [file for file in files if index in file]
conti = []
for band_number in range(start_index,end_index+1,1):
    # create daily vrts and extract

    for i, tif in enumerate(index_files):
        output = trash + tif.split('/')[-1].split('.')[0] + '_' + str(band_number) + '_' + str(i) + '.tif'
        gdal.Translate(output, tif, bandList=[band_number])

    # Now build VRT
    vrt_options = gdal.BuildVRTOptions(separate=False)
    vrt_ds = gdal.BuildVRT(f'{temp}{year}_{index}_{band_number}.vrt', getFilelist(trash, '.tif'), options=vrt_options)
    # vrt_ds.FlushCache()
    # vrt = gdal.Open(f'{temp}{year}_{index}_{band_number}.vrt')
    warpi = warp_to_template(vrt_ds, 
            '/data/Aldhani/eoagritwin/et/Auxiliary/S3_tif_GER_maks/powermask.tif',
            mask_path='/data/Aldhani/eoagritwin/et/Auxiliary/S3_tif_GER_maks/powermask.tif',
            #outPath=f'/data/Aldhani/eoagritwin/et/FORCE/vrt_dumps/{year}_{index}_{band_number}.tif',
            outType=gdal.GDT_Int16)
    conti.append(warpi)
    [os.remove(file) for file in getFilelist(trash, '.tif')]

In [ ]:
vals.append(warpi[rows,cols])
arr = np.stack(vals)
mask = np.all(arr == -9999, axis=0)
small_arr = arr[:, ~mask]

np.save(f'/data/{origin}et/Auxiliary/dumps_for_training_collecting/{year}_{index}_X0063_Y0044_arr10', small_arr)
nested_masked = [row_col for row_col, valid in zip(nested, ~mask) if valid]
np.save(f'/data/{origin}et/Auxiliary/dumps_for_training_collecting/{year}_{index}_X0063_Y0044_rowcols10', nested_masked)'

In [5]:
dat = np.load(f'/data/{origin}et/Auxiliary/dumps_for_training_collecting/2018_NDM.npy')


In [10]:
a=9542
b=9600
print(nested[a:b])
print(f'ROWS: {rows[a:b]}  COLS: {cols[a:b]}')

[['141', '846'], ['141', '847'], ['141', '848'], ['141', '849'], ['141', '851'], ['141', '852'], ['141', '853'], ['141', '854'], ['141', '855'], ['141', '856'], ['141', '859'], ['141', '860'], ['141', '861'], ['141', '863'], ['141', '864'], ['141', '865'], ['141', '866'], ['141', '867'], ['141', '870'], ['141', '871'], ['141', '875'], ['141', '876'], ['141', '877'], ['141', '878'], ['141', '881'], ['141', '882'], ['141', '883'], ['141', '885'], ['141', '886'], ['141', '888'], ['141', '889'], ['141', '890'], ['141', '891'], ['141', '892'], ['141', '893'], ['141', '894'], ['141', '895'], ['141', '896'], ['141', '897'], ['141', '898'], ['141', '899'], ['141', '900'], ['141', '901'], ['141', '902'], ['141', '905'], ['141', '906'], ['141', '907'], ['141', '908'], ['141', '909'], ['141', '910'], ['141', '911'], ['141', '913'], ['141', '914'], ['142', '346'], ['142', '347'], ['142', '348'], ['142', '349'], ['142', '350']]
ROWS: [141, 141, 141, 141, 141, 141, 141, 141, 141, 141, 141, 141, 141,

In [ ]:
force_path = '/data/Aldhani/eoagritwin/force/output/S3/2018/X0063_Y0044/TSI/'
files = getFilelist(force_path, '.tif')
# delete all other files
[RasterKiller(file) for file in files if str(year) not in file.split('_')[-1]]
files = getFilelist(force_path, '.tif')
files_annual = [file for file in files if str(year) in file]
indices = list(set([file_annual.split('_')[-3] for file_annual in files_annual]))
indices.sort()

In [ ]:
print(rc100[0:10,5])
print(rc10[0:10,5])

In [ ]:
ds = gdal.Open(index_file,0)
# Get list of all metadata domains
domains = ds.GetMetadataDomainList()

# Collect metadata from all domains
metadata_all = {}

for domain in domains:
    metadata = ds.GetMetadata(domain)
    metadata_all[domain] = metadata
    print(f"--- Metadata for domain: {domain} ---")
    for key, value in metadata.items():
        print(f"{key}: {value}")
    print()

band = ds.GetRasterBand(1)
# Band-specific metadata
print("Band Metadata:", band.GetMetadata())

# Band-specific statistics and other info
print("Min/Max:", band.GetMinimum(), band.GetMaximum())
print("Scale:", band.GetScale())       # for scaled values
print("Offset:", band.GetOffset())
print("Unit Type:", band.GetUnitType())
print("NoData Value:", band.GetNoDataValue())

In [ ]:
arr = ds.GetRasterBand(1).ReadAsArray()
# Replace nodata with NaN
arr = np.where(arr == -9999, np.nan, arr)

# Invert the log10 transform
real_amplitude = 10 ** (arr-4)

In [ ]:
print(np.unique(warpi))

plt.imshow(warpi[300:335,580:640])
plt.axis('off')  # Hide axis for cleaner image display

plt.show()

In [ ]:
# Build a dict of dicts: row_col → {year: value}
data_dict = {rc: {} for rc in row_col_list}

for year, values in yearly_values.items():
    for rc, val in zip(row_col_list, values):
        data_dict[rc][year] = val